**1. Install req packages**

In [2]:
!pip -q install langchain langchain_community pypdf huggingface_hub
!pip -q install bitsandbytes accelerate transformers
!pip -q install datasets loralib sentencepiece
!pip -q install pypdf
!pip -q install sentence_transformers
!pip -q install unstructured
!pip install tokenizers
!pip install xformers
!pip install pinecone-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.7/409.7 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour 

**2. Import All Required lib**

In [3]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.chains import RetrievalQA
from langchain import HuggingFacePipeline
from huggingface_hub import notebook_login
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import sys
import os
import textwrap

In [4]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

**3: Pass the URLs and extract the data from these URLs**

In [5]:
URLS =[
    'https://malavikagowthaman.github.io/Malavika-Gowthaman-Portfolio/'
]

In [6]:
loaders = UnstructuredURLLoader(urls=URLS)
data =loaders.load()
data

[Document(metadata={'source': 'https://malavikagowthaman.github.io/Malavika-Gowthaman-Portfolio/'}, page_content="Hi, I'm Malavika AI Engineer\n\nDownload CV\n\nAbout\n\nI am Malavika Gowthaman\n\nAI Engineer and Data Scientist with 5 years of experience in data collection, pre-processing, model development and deployment. Expert in Deep Learning, Neural Networks, Machine Learning, and LLMs, with a strong background in IT, Computer Science, mathematics, Software Engineering and data analytics. Skilled in Power BI, Tableau, SQL, ETL Process, ML Flow and Python.\n\nWork Experience\n\nHope Artificial Intelligence Pvt Ltd - AI Engineer\n\n04/24 – Present\n\nDeveloping a healthcare app to improve disease diagnosis accuracy by 20% through data collection and analysis of over 10,000 patient records. Implementing machine learning models for predictive analysis, aiming to reduce diagnostic time by 30%. Focusing on optimizing performance and providing real-time insights for better decision-makin

In [7]:
len(data)

1

**4. Split the text into chunks**

In [8]:
text_splitter = CharacterTextSplitter(separator = '\n',
                                      chunk_size =1000,
                                      chunk_overlap = 200)

In [9]:
text_chunks = text_splitter.split_documents(data)

In [10]:
len(text_chunks)

4

In [11]:
text_chunks[0]

Document(metadata={'source': 'https://malavikagowthaman.github.io/Malavika-Gowthaman-Portfolio/'}, page_content="Hi, I'm Malavika AI Engineer\nDownload CV\nAbout\nI am Malavika Gowthaman\nAI Engineer and Data Scientist with 5 years of experience in data collection, pre-processing, model development and deployment. Expert in Deep Learning, Neural Networks, Machine Learning, and LLMs, with a strong background in IT, Computer Science, mathematics, Software Engineering and data analytics. Skilled in Power BI, Tableau, SQL, ETL Process, ML Flow and Python.\nWork Experience\nHope Artificial Intelligence Pvt Ltd - AI Engineer\n04/24 – Present\nDeveloping a healthcare app to improve disease diagnosis accuracy by 20% through data collection and analysis of over 10,000 patient records. Implementing machine learning models for predictive analysis, aiming to reduce diagnostic time by 30%. Focusing on optimizing performance and providing real-time insights for better decision-making in healthcare."

**5. Download Hugging face embeddings**

In [12]:
embedding = HuggingFaceEmbeddings()

<ipython-input-12-ec4b3f508956>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings()
<ipython-input-12-ec4b3f508956>:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding = HuggingFaceEmbeddings()
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingf

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
embedding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-mpnet-base-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [14]:
query_result = embedding.embed_query("Hello")
len(query_result)

768

In [15]:
query_result

[0.03063981980085373,
 -0.006230128463357687,
 -0.002121545607224107,
 0.01387915387749672,
 0.02648673765361309,
 0.004065841902047396,
 -0.0038712231907993555,
 0.0352698415517807,
 0.01148827001452446,
 -0.0038065058179199696,
 0.013236366212368011,
 -0.011885667219758034,
 -0.00016039602633100003,
 0.02157069556415081,
 0.03014967031776905,
 -0.1055913195014,
 0.03541698679327965,
 0.006596237421035767,
 -0.055303916335105896,
 -0.00022675195941701531,
 -0.007831449620425701,
 -0.0047395071014761925,
 -2.19344838114921e-05,
 0.037050455808639526,
 0.022426944226026535,
 0.004295846447348595,
 -0.0010749970097094774,
 0.007416189648211002,
 0.011647170409560204,
 0.019009580835700035,
 -0.011946497485041618,
 0.0021739352960139513,
 0.026941750198602676,
 0.016080887988209724,
 2.160505573556293e-06,
 -0.008001381531357765,
 -0.013405383564531803,
 0.016731996089220047,
 -0.009559356607496738,
 -0.02241317555308342,
 -0.00016317659174092114,
 -0.0004943417152389884,
 -0.009095584973

**06: Convert the Text Chunks into Embeddings and Create a Knowledge Base**

In [26]:
!pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.7 MB/s eta 0:00:00


In [27]:
from langchain.vectorstores import FAISS

In [28]:
vector_store = FAISS.from_documents(text_chunks,embedding)

In [29]:
vector_store

**07: Create a Large Language Model (LLM) Wrapper**

In [30]:
notebook_login()

In [31]:
model = "meta-llama/Llama-2-7b-chat-hf"

In [32]:
tokenizer = AutoTokenizer.from_pretrained(model,
                                          use_auth_token=True)

model = AutoModelForCausalLM.from_pretrained(model,
                                             use_auth_token =True,
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             load_in_8bit=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:809: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [33]:
pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [34]:
llm = HuggingFacePipeline(pipeline=pipe,model_kwargs={'temperature':0})

<ipython-input-34-1d68943b039d>:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe,model_kwargs={'temperature':0})


In [35]:
llm.predict("What is the meaning of life?")

<ipython-input-35-92d54fbf0b52>:1: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm.predict("What is the meaning of life?")


'What is the meaning of life? This is a question that has puzzled philosophers, theologians, scientists, and many other thinkers throughout history. Here are some possible answers:\n\n1. Religious or spiritual beliefs: Many people believe that the meaning of life is to fulfill a divine or spiritual purpose. According to this view, life has a higher purpose or goal that is connected to a deity or a spiritual realm. The purpose of life may be to follow God’s will, to achieve spiritual enlightenment, or to fulfill a specific mission or calling.\n2. Personal fulfillment: Some people believe that the meaning of life is to find personal fulfillment and happiness. According to this view, life is all about pursuing one’s passions and interests, and experiencing joy, pleasure, and satisfaction. The meaning of life is to live in a way that brings happiness and fulfillment to oneself and those around us.\n3. Social or cultural values: Others believe that the meaning of life is to contribute to so

**08: Initialize the Retrieval QA with Source Chain**

In [36]:
from langchain.chains import RetrievalQA

In [38]:
docs = vector_store.similarity_search(query, k=3)
docs

[Document(metadata={'source': 'https://malavikagowthaman.github.io/Malavika-Gowthaman-Portfolio/'}, page_content="Hi, I'm Malavika AI Engineer\nDownload CV\nAbout\nI am Malavika Gowthaman\nAI Engineer and Data Scientist with 5 years of experience in data collection, pre-processing, model development and deployment. Expert in Deep Learning, Neural Networks, Machine Learning, and LLMs, with a strong background in IT, Computer Science, mathematics, Software Engineering and data analytics. Skilled in Power BI, Tableau, SQL, ETL Process, ML Flow and Python.\nWork Experience\nHope Artificial Intelligence Pvt Ltd - AI Engineer\n04/24 – Present\nDeveloping a healthcare app to improve disease diagnosis accuracy by 20% through data collection and analysis of over 10,000 patient records. Implementing machine learning models for predictive analysis, aiming to reduce diagnostic time by 30%. Focusing on optimizing performance and providing real-time insights for better decision-making in healthcare.

In [39]:
qa = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=vector_store.as_retriever())

In [40]:
query = "What is malavika gowthaman experience?"
qa.run(query)

<ipython-input-40-6584f8e4c0a8>:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qa.run(query)


"Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nHi, I'm Malavika AI Engineer\nDownload CV\nAbout\nI am Malavika Gowthaman\nAI Engineer and Data Scientist with 5 years of experience in data collection, pre-processing, model development and deployment. Expert in Deep Learning, Neural Networks, Machine Learning, and LLMs, with a strong background in IT, Computer Science, mathematics, Software Engineering and data analytics. Skilled in Power BI, Tableau, SQL, ETL Process, ML Flow and Python.\nWork Experience\nHope Artificial Intelligence Pvt Ltd - AI Engineer\n04/24 – Present\nDeveloping a healthcare app to improve disease diagnosis accuracy by 20% through data collection and analysis of over 10,000 patient records. Implementing machine learning models for predictive analysis, aiming to reduce diagnostic time by 30%. Focusing on optimizing performance and providing real-ti

In [41]:
while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit()
  if user_input == '':
    continue
  result = qa({'query': user_input})
  print(f"Answer: {result['result']}")

Input Prompt: contact number


<ipython-input-41-c213a85929f6>:8: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa({'query': user_input})


Answer: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Projects
Skills
Python
Machine Learning
Deep Learning
NLP
Large Language Models
Data Science
Generative AI
Data Analytics
Data Collection
Feature Engineering
SQL
Tableau
Power BI
Talend
Informatica Power Center
Azure ML Studio
Auto ML
Amazon Web Service
Google Cloud Platform
Visual Studio Code
Anaconda
Google Colab
Jupyter Notebook
Download CV
Get In Touch
Name: Malavika Gowthaman
Phone: +1 (647) 446-1531
Email: malavikag11@gmail.com
Address: Toronto, ON, Canada

Hi, I'm Malavika AI Engineer
Download CV
About
I am Malavika Gowthaman
AI Engineer and Data Scientist with 5 years of experience in data collection, pre-processing, model development and deployment. Expert in Deep Learning, Neural Networks, Machine Learning, and LLMs, with a strong background in IT, Computer Science, mathematics, Software Engineering and dat

SystemExit: 

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
